In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

크로스 벨리데이션(Cross Validation)  
앞선 지도 학습 알고리즘에서는 전체 데이터를 트레이닝 데이터와 테스트 데이터로 나누어 모델을 학습시켰다. 이번에는 하이퍼파라미터 튜닝을 위해 클로스 벨리데이션 기법을 활용해 본다.

<table align="left" width="700">
    <tr>
        <td colspan="6" style="border: 1px solid;">
            <div style="text-align: center">Total Data</div>
        </td>
    </tr>
    <tr>
        <td colspan="5" style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Test</div>
        </td>
    </tr>
    <tr>
        <td>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td rowspan="5" style="border: 1px solid;">
            <div style="text-align: center">
                파라미터,<br>
                하이퍼파라미터<br>
                설정
            </div>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
    </tr>
    <tr>
        <td>
        </td>
    </tr>
    <tr>
        <td colspan="5">
            <div>모형평가</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Test</div>
        </td>
    </tr>
</table>

In [2]:
# 데이터 불러오기
from sklearn import datasets # 사이킷런이 제공하는 데이터셋을 사용하기 위해 import 한다.
raw_wine = datasets.load_wine() # 와인 데이터를 불러온다.

In [3]:
# 피쳐, 타겟 데이터 저장
X = raw_wine.data # 피쳐 데이터를 저장한다.
y = raw_wine.target # 타겟 데이터를 저장한다.
X.shape

(178, 13)

In [4]:
# 트레이닝, 테스트 데이터 분할
from sklearn.model_selection import train_test_split # 트레이닝, 테스트 데이터 분할을 위해 import 한다.
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)
print(X_tn.shape, y_tn.shape, X_te.shape, y_te.shape) # 트레이닝 데이터와 테스트 데이터로 분할한다.

(133, 13) (133,) (45, 13) (45,)


In [5]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위해 import 한다.
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
X_tn_std = std_scale.fit_transform(X_tn) # 트레이닝 데이터를 적합 후 표준화 한다.
X_te_std = std_scale.transform(X_te) # 테스트 데이터를 표준화 한다.

In [6]:
# 모델 생성 후 데이터 학습
from sklearn.svm import SVC  # 서포트 벡터 머신 알고리즘을 사용하기 위해 import 한다.
# StratifiedKFold은 일반적인 K-Fold 크로스 벨리데이션과는 다르게 라벨링 비율을 유지하면서 데이터를 추출한다.
from sklearn.model_selection import StratifiedKFold # 크로스 벨리데이션을 사용하기 위해 import 한다.
# 그리드 서치(Grid Search)
# 그리드 서치는 사용자가 지정해준 몇 가지 잠재적인 파라미터들의 후보군들의 조합 중에서 가장 beat 조합을 찾아준다.
# 사이킷런은 그리드 서치를 제공하므로 사용자가 파라미터들의 후보군들을 하나하나 대입하 가면서 loss를 확인하는 작업을
# 그리드 서치가 대신 해주므로 손쉽게 사용할 수 있다.
# 하이퍼파라미터 후보군의 갯수만큼 비례해서 시간이 늘어나 최적의 조합을 찾는 시간이 매우 오래걸리는 단점이 있다.
from sklearn.model_selection import GridSearchCV # 그리드 서치를 사용하기 위해 import 한다.

clf_svm = SVC() # 서포트 벡터 머신 모델을 만든다.
# 그리드 서치에 사용할 하이퍼파라미터 후보군을 설정한다.
param_grid = {
    'kernel': ('linear', 'rbf'), # 서포트 벡터 머신에서 커널은 정확도가 높은 linear 또는 rbf로 설정한다.
    'C': [0.5, 1, 10, 100]
}
# n_splits 옵션으로 트레이닝 데이터를 나눌 개수를 지정하고 shuffle=True 옵션을 지정하면 데이터를 섞는다.
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0) # 크로스 벨리데이션에 사용할 객체를 만든다.

# clf_svm.fit(X_tn_std, y_tn) # 그리드 서치와 크로스 벨리데이션을 적용하지 않은 서포트 벡터 머신 알고리즘 학습
# 그리드 서치 객체를 만든다.
# estimator 옵션으로 학습할 모델을 지정하고 param_grid 옵션으로 하이퍼파라미터 후보군을 지정한다.
# cv 옵션으로 크로스 벨리데이션 객체를 지정하고 scoring 옵션으로 모델 평가 방법을 지정한다.
grid_cv = GridSearchCV(estimator=clf_svm, param_grid=param_grid, cv=kfold, scoring='accuracy')
# 표준화된 트레이닝 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn를 모델에 넣어서 그리드 서치와 크로스 벨리데이션을 
# 적용한 서포트 벡터 머신 알고리즘을 학습시킨다.
grid_cv.fit(X_tn_std, y_tn)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [0.5, 1, 10, 100], 'kernel': ('linear', 'rbf')},
             scoring='accuracy')

In [7]:
# 그리드 서치 결과 확인
# 그리드 서치 객체에서 cv_results_ 속성으로 그리드 서치 결과(설정한 파라미터, 모델 평가 등)를 확인할 수 있다.
print(grid_cv.cv_results_)

{'mean_fit_time': array([0.00100026, 0.00079975, 0.00100021, 0.00080032, 0.        ,
       0.00060005, 0.00100036, 0.00100045]), 'std_fit_time': array([8.47644274e-07, 3.99876010e-04, 7.74768677e-07, 4.00161772e-04,
       0.00000000e+00, 4.89940409e-04, 6.97552626e-07, 8.03580262e-07]), 'mean_score_time': array([0.        , 0.00040016, 0.        , 0.00040011, 0.        ,
       0.00060019, 0.00020018, 0.00059996]), 'std_score_time': array([0.        , 0.0004901 , 0.        , 0.00049004, 0.        ,
       0.00049006, 0.00040035, 0.00048986]), 'param_C': masked_array(data=[0.5, 0.5, 1, 1, 10, 10, 100, 100],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf', 'linear', 'rbf',
                   'linear', 'rbf'],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params':

In [8]:
import pandas as pd
pd.DataFrame(grid_cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.0010,8.476443e-07,0.0000,0.00000,0.5,linear,"{'C': 0.5, 'kernel': 'linear'}",0.888889,0.962963,0.925926,1.000000,0.846154,0.924786,0.054014,5
1,0.0008,3.998760e-04,0.0004,0.00049,0.5,rbf,"{'C': 0.5, 'kernel': 'rbf'}",0.962963,1.000000,0.962963,0.961538,1.000000,0.977493,0.018384,1
2,0.0010,7.747687e-07,0.0000,0.00000,1,linear,"{'C': 1, 'kernel': 'linear'}",0.888889,0.962963,0.925926,1.000000,0.846154,0.924786,0.054014,5
3,0.0008,4.001618e-04,0.0004,0.00049,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.925926,0.962963,0.962963,0.961538,1.000000,0.962678,0.023431,2
4,0.0000,0.000000e+00,0.0000,0.00000,10,linear,"{'C': 10, 'kernel': 'linear'}",0.888889,0.962963,0.925926,1.000000,0.846154,0.924786,0.054014,5
5,0.0006,4.899404e-04,0.0006,0.00049,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.925926,0.962963,0.962963,0.961538,1.000000,0.962678,0.023431,2
6,0.0010,6.975526e-07,0.0002,0.00040,100,linear,"{'C': 100, 'kernel': 'linear'}",0.888889,0.962963,0.925926,1.000000,0.846154,0.924786,0.054014,5
7,0.0010,8.035803e-07,0.0006,0.00049,100,rbf,"{'C': 100, 'kernel': 'rbf'}",0.925926,0.962963,0.962963,0.961538,1.000000,0.962678,0.023431,2


In [9]:
import numpy as np
np.transpose(pd.DataFrame(grid_cv.cv_results_))

,0,1,2,3,4,5,6,7
mean_fit_time,0.001,0.0008,0.001,0.0008,0.0,0.0006,0.001,0.001
std_fit_time,0.000001,0.0004,0.000001,0.0004,0.0,0.00049,0.000001,0.000001
mean_score_time,0.0,0.0004,0.0,0.0004,0.0,0.0006,0.0002,0.0006
std_score_time,0.0,0.00049,0.0,0.00049,0.0,0.00049,0.0004,0.00049
param_C,0.5,0.5,1,1,10,10,100,100
param_kernel,linear,rbf,linear,rbf,linear,rbf,linear,rbf
params,"{'C': 0.5, 'kernel': 'linear'}","{'C': 0.5, 'kernel': 'rbf'}","{'C': 1, 'kernel': 'linear'}","{'C': 1, 'kernel': 'rbf'}","{'C': 10, 'kernel': 'linear'}","{'C': 10, 'kernel': 'rbf'}","{'C': 100, 'kernel': 'linear'}","{'C': 100, 'kernel': 'rbf'}"
split0_test_score,0.888889,0.962963,0.888889,0.925926,0.888889,0.925926,0.888889,0.925926
split1_test_score,0.962963,1.0,0.962963,0.962963,0.962963,0.962963,0.962963,0.962963
split2_test_score,0.925926,0.962963,0.925926,0.962963,0.925926,0.962963,0.925926,0.962963


In [10]:
# best 스코어와 best 스코어를 얻어낸 하이퍼파라미터를 확인한다.
print(grid_cv.best_score_) # 그리드 서치 결과 중 best 스코어를 얻어온다.
print(grid_cv.best_params_) # 그리드 서치 결과 중 best 스코어를 얻어낸 하이퍼파라미터를 얻어온다.

0.9774928774928775
{'C': 0.5, 'kernel': 'rbf'}


In [11]:
# best 스코어를 얻어낸 하이퍼파라미터를 사용한 모델을 얻어와서 최종 모델로 설정한다.
clf_grid = grid_cv.best_estimator_
print(clf_grid)

SVC(C=0.5)


In [12]:
# 크로스 벨리데이션 스코어 확인
from sklearn.model_selection import cross_validate # 크로스 벨리데이션 스코어를 확인하기 위해 import 한다.
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
cv_score = cross_validate(clf_grid, X_tn_std, y_tn, cv=kfold, scoring=metrics)
for key in cv_score:
    print(key, cv_score[key])

fit_time [0.00200057 0.00200105 0.00100064 0.0010004  0.00100017]
score_time [0.00299978 0.00200057 0.00200033 0.00200009 0.00200415]
test_accuracy [0.96296296 1.         0.96296296 0.96153846 1.        ]
test_precision_macro [0.96296296 1.         0.96969697 0.96969697 1.        ]
test_recall_macro [0.96666667 1.         0.96296296 0.95833333 1.        ]
test_f1_macro [0.9628483  1.         0.96451914 0.96190476 1.        ]


In [13]:
# 크로스 벨리데이션 스코어 확인
from sklearn.model_selection import cross_val_score # 크로스 벨리데이션 스코어를 확인하기 위해 import 한다.
# cross_validate() 메소드는 scoring 옵션에 여러개의 지표를 동시에 넣을 수 있지만 cross_val_score() 메소드는
# scoring 옵션에 딱 1개의 지표만 넣을 수 있다.
# cross_validate() 메소드는 여러개의 지표를 동시에 확인할 때 사용하고 cross_val_score() 메소드는 한 가지
# 지표만 확인할 때 사용한다.
cv_score = cross_val_score(clf_grid, X_tn_std, y_tn, cv=kfold, scoring='accuracy')
print(cv_score) # 각 split 별 정확도를 확인
print(cv_score.mean())
print(cv_score.std())

[0.96296296 1.         0.96296296 0.96153846 1.        ]
0.9774928774928775
0.01838434849561446


In [14]:
# 테스트 데이터 예측
# 표준화된 테스트 데이터 X_te_std로 예측한다.
pred_svm = clf_grid.predict(X_te_std)
print(pred_svm)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


In [15]:
# 정확도 평가
from sklearn.metrics import accuracy_score # 모델의 정확도를 평가하기 위해 import 한다.
# accuracy_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_svm)를 넘겨 정확도를 계산한다.
accuracy = accuracy_score(y_te, pred_svm)
print(accuracy)

1.0


In [16]:
# 정밀도 평가
from sklearn.metrics import precision_score # 모델의 정밀도를 평가하기 위해 import 한다.
# precision_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_svm)를 넘겨 정밀도를 계산한다.
precision = precision_score(y_te, pred_svm, average=None)
print(precision)

[1. 1. 1.]


In [17]:
# 재현율 평가
from sklearn.metrics import recall_score # 모델의 재현율를 평가하기 위해 import 한다.
# recall_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_svm)를 넘겨 재현율를 계산한다.
recall = recall_score(y_te, pred_svm, average=None)
print(recall)

[1. 1. 1.]


In [18]:
# f1 score 평가
from sklearn.metrics import f1_score # 모델의 f1 score를 평가하기 위해 import 한다.
# f1_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_svm)를 넘겨 f1 score를 계산한다.
f1 = f1_score(y_te, pred_svm, average=None)
print(f1)

[1. 1. 1.]


In [19]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix # 혼돈 행렬을 만들기 위해 import 한다.
# confusion_matrix() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_svm)를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_svm)
print(conf_matrix)

[[16  0  0]
 [ 0 21  0]
 [ 0  0  8]]


In [20]:
# 분류 리포트 확인
from sklearn.metrics import classification_report # 분류 리포트를 출력하기 위해 import 한다.
# classification_report() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_svm)를 넘겨 분류 리포트를 만든다.
target_names = raw_wine.target_names
class_report = classification_report(y_te, pred_svm, target_names=target_names)
print(class_report)

              precision    recall  f1-score   support

     class_0       1.00      1.00      1.00        16
     class_1       1.00      1.00      1.00        21
     class_2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

